# Pruning the merged rough df

In [1]:
import pandas as pd
import numpy as np
from dutchanalyzer.config import *
from dutchanalyzer.utils import *
from pathlib import Path
from dotenv import load_dotenv
import datetime
import re

In [2]:
current_save_path = Path(INTERIM_DATA_DIR, 'preprocessing', 'wikt', '30-10-25')
exploration_save_path = Path(INTERIM_DATA_DIR, 'exploration', '2025-10-30')
rough_merged_path = Path(exploration_save_path, 'rough_merged_30-10-25.csv')

In [3]:
rough_df = pd.read_csv(rough_merged_path)

C:\Users\elise\AppData\Local\Temp\ipykernel_15972\3205766320.py:1: DtypeWarning: Columns (5,6,14,16,19,20,22,26,27,29,30,31,33,34,35,36,39,41,43,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  rough_df = pd.read_csv(rough_merged_path)


In [7]:
rough_df.columns

Index(['Unnamed: 0', 'word', 'pos', 'lang_code', 'senses', 'abbreviations',
       'anagrams', 'antonyms', 'categories', 'coordinate_terms', 'derived',
       'descendants', 'etymology_number', 'etymology_templates',
       'etymology_text', 'etymology_texts', 'form_of', 'forms',
       'head_templates', 'holonyms', 'homophones', 'hypernyms', 'hyphenation',
       'hyphenations', 'hyponyms', 'inflection_templates', 'info_templates',
       'instances', 'lang', 'meronyms', 'metonyms', 'notes', 'origin',
       'original_title', 'paronyms', 'pos_title', 'proverbs', 'related',
       'sounds', 'source', 'synonyms', 'tags', 'translations', 'troponyms',
       'wikidata', 'wikipedia', 'invalid', 'lower', 'word_id', 'alt_of',
       'redirects'],
      dtype='object')

## Group Definitions

In [ ]:
# def transform_df_to_col_table(origin_df, col_name, group_key='tmp_id'):
#     dropped_df = origin_df[~origin_df[col_name].isna()]
#     words_df = dropped_df.loc[:, ['tmp_id', 'word']]
#     df = dropped_df.loc[:, ['tmp_id', 'word_id', 'pos', col_name]]
#     df[col_name] = df[col_name].apply(safe_eval)
#     df = df.explode(col_name, ignore_index=True)
    
#     df = pd.concat([df.drop(columns=col_name), pd.json_normalize(df[col_name])],axis=1)
    
#     df = df.groupby(group_key, as_index=False).aggregate('first').reindex(columns=df.columns)
#     if 'word' in df.columns:
#         df = df.rename(columns={'word': col_name})
    
#     df = df.merge(words_df, on='tmp_id')
#     df_cols = df.columns.tolist()
#     df_cols = df_cols[0:2] + ['word'] + df_cols[2:-1]
#     df = df[df_cols]
    
#     return df

In [20]:
def cut_df_to_category(df, col_name, cols_to_keep=['word_id', 'word', 'pos'], save_location=current_save_path, parent_category="", subparent_category='', supress_save=False):
    new_df = df[~df[col_name].isna()]
    cols = cols_to_keep + [col_name]
    new_df = new_df.loc[:, cols]
    if supress_save != True:
        if parent_category != "":
            file_path = "{}/{}/{}_{}.csv".format(save_location, parent_category, parent_category, col_name)
        else:
            file_path = "{}/{}.csv".format(save_location, col_name)
        new_df.to_csv(file_path)
    return new_df

In [193]:
def process_tags_text_note_cols(df, parent_category, subparent_category=''):
    cols = df.columns
    note_df = pd.DataFrame()
    tag_df = pd.DataFrame()
    text_df = pd.DataFrame()
    concat_list = []
    concat_type_dict = {}
    cols_using = []
    if 'note' in cols:
        note_df = cut_df_to_category(df,'note', parent_category=parent_category, supress_save=True)
        concat_list.append(note_df)
        concat_type_dict['note'] = 'object'
        cols_using.append('note')
    if 'tags' in cols:
        tag_df = cut_df_to_category(df, 'tags', parent_category=parent_category, supress_save=True)
        concat_list.append(tag_df)
        concat_type_dict['tags'] = 'object'
        cols_using.append('tags')
    if 'text' in cols:
        text_df = cut_df_to_category(df, 'text', parent_category=parent_category, supress_save=True)
        concat_list.append(text_df)
        concat_type_dict['text'] = 'object'
        cols_using.append('text')
    
    if concat_list == []:
        print('no text, tags, or note cols')
        return None
    combined_df = pd.concat(concat_list, ignore_index=True)
    combined_df.astype(concat_type_dict).dtypes

    combined_ntt_col = []
    for i, row in combined_df.iterrows():
        row_values = []
        note = None
        tags = None
        text = None
        for k in concat_type_dict.keys():
            
            if k == 'note':
                note = row['note']
                if type(note) != float:
                    row_values.append({'note': note})
            if k == 'tags':
                tags = row['tags']
                if type(tags) != float:
                    row_values.append({'tags': tags})
            if k == 'text':
                text = row['text']
                if type(text) != float:
                    row_values.append({'text': text})
        
        combined_ntt_col.append(row_values)
    combined_df['text_tags_notes'] = combined_ntt_col
    combined_df = combined_df.drop(columns=cols_using)
    # f = "{}_{}.csv".format(parent_category, 'text_tags_notes')
    # if subparent_category != '':
    #     f = "{}_{}_{}.csv".format(parent_category, subparent_category, 'text-tags-notes')
    # combined_df.to_csv(Path(category_df_dir, parent_category, f))
    return combined_df

In [9]:
rough_df.columns

Index(['Unnamed: 0', 'word', 'pos', 'lang_code', 'senses', 'abbreviations',
       'anagrams', 'antonyms', 'categories', 'coordinate_terms', 'derived',
       'descendants', 'etymology_number', 'etymology_templates',
       'etymology_text', 'etymology_texts', 'form_of', 'forms',
       'head_templates', 'holonyms', 'homophones', 'hypernyms', 'hyphenation',
       'hyphenations', 'hyponyms', 'inflection_templates', 'info_templates',
       'instances', 'lang', 'meronyms', 'metonyms', 'notes', 'origin',
       'original_title', 'paronyms', 'pos_title', 'proverbs', 'related',
       'sounds', 'source', 'synonyms', 'tags', 'translations', 'troponyms',
       'wikidata', 'wikipedia', 'invalid', 'lower', 'word_id', 'alt_of',
       'redirects'],
      dtype='object')

In [5]:
rough_df['invalid'].value_counts()

invalid
False    2637038
Name: count, dtype: int64

In [ ]:
rough_df.head()

pos
noun           1476278
verb            452245
adj             336439
name            289019
adv              43583
phrase            8101
intj              7903
prep_phrase       4817
prefix            3824
num               2580
suffix            2576
proverb           2455
pron              1895
prep              1649
contraction       1176
conj               730
det                697
symbol             346
character          170
particle           149
abbrev             123
article             77
infix               77
interfix            71
circumfix           24
circumpos           23
postp                6
affix                2
adv_phrase           2
punct                1
Name: count, dtype: int64
lang_code
en    2121885
nl     515153
Name: count, dtype: int64
lang
English       2113176
Nederlands     304614
Dutch          210539
Engels           8709
Name: count, dtype: int64
original_title
Unsupported titles/d:o    1
Name: count, dtype: int64


In [ ]:
rough_df.drop(columns=['original_title', 'invalid', 'Unnamed: 0'], inplace=True)
rough_df.to_csv(Path(current_save_path, 'rough_df_temp_processing_15uur_30-10-25.csv'))

## Exploring Value Counts

In [ ]:
rough_df_temp_path = Path(current_save_path, 'rough_df_temp_processing_15uur_30-10-25.csv')
rough_df = pd.read_csv(rough_df_temp_path, index_col=0)

C:\Users\elise\AppData\Local\Temp\ipykernel_19288\1237705882.py:2: DtypeWarning: Columns (5,6,14,16,19,20,22,26,27,29,30,31,33,34,35,38,40,42,43,45,47) have mixed types. Specify dtype option on import or set low_memory=False.
  rough_df = pd.read_csv(rough_df_temp_path)


In [5]:
cols = ['word', 'word_id', 'pos', 'lang_code', 'senses', 'abbreviations', 'alt_of', 
       'anagrams', 'antonyms', 'categories', 'coordinate_terms', 'derived',
       'descendants', 'etymology_number', 'etymology_templates',
       'etymology_text', 'etymology_texts', 'form_of', 'forms',
       'head_templates', 'holonyms', 'homophones', 'hypernyms', 'hyphenation',
       'hyphenations', 'hyponyms', 'inflection_templates', 'info_templates',
       'instances', 'lang', 'meronyms', 'metonyms', 'notes', 'origin',
       'paronyms', 'pos_title', 'proverbs', 'redirects','related', 'sounds', 'source',
       'synonyms', 'tags', 'translations', 'troponyms', 'wikidata',
       'wikipedia', 'lower']

In [6]:
# check if there are any proper duplicates
dups = rough_df.duplicated()

In [ ]:
rough_df['tmpid'] = [i for i in range(len(rough_df))]

In [39]:
rough_df.rename(columns={'temp_num': 'tmpid'}, inplace=True)

In [52]:
val_counts = rough_df['lower'].value_counts()
words = val_counts.index.to_list()

In [53]:
words = [w.strip() for w in words]
val_df = pd.DataFrame(val_counts)

In [55]:
val_df.reset_index(inplace=True)

In [57]:
display(val_df)

,lower,count
0,a,18
1,mat,17
2,o,16
3,min,13
4,d,13
...,...,...
1018825,étuis,1
1018826,évolués,1
1018827,carty,1
1018828,éénvormig,1


In [62]:
rough_df = rough_df.merge(val_counts, how='outer', on='lower')

In [81]:
rough_df

,Unnamed: 0,word,pos,lang_code,senses,abbreviations,anagrams,antonyms,categories,coordinate_terms,...,wikidata,wikipedia,lower,word_id,alt_of,redirects,temp_entry_num,tmpid,count,ety_text
0,0,',symbol,en,"[{'links': [['’', '’#English']], 'glosses': ['...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,',0,NaN,NaN,0,0,1.0,NaN
1,23,'a,verb,en,"[{'links': [['a', 'a#English']], 'raw_glosses'...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,'a,1,NaN,NaN,23,23,1.0,NaN
2,24,'a'a,noun,en,"[{'links': [['aa', 'aa#English']], 'glosses': ...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,'a'a,3,NaN,NaN,24,24,1.0,NaN
3,25,'akikikis,noun,en,"[{'links': [[""'akikiki"", ""'akikiki#English""]],...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,'akikikis,9,NaN,NaN,25,25,1.0,NaN
4,26,'alf,noun,en,"[{'links': [['Pronunciation spelling', 'pronun...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,'alf,10,NaN,NaN,26,26,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2637033,2637033,übersexuals,noun,en,"[{'links': [['übersexual', 'übersexual#English...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1644307,NaN,NaN,2637033,2637033,NaN,NaN
2637034,2637034,ümlaut,noun,en,[{'examples': [{'text': 'The Traveler said he ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1644308,NaN,NaN,2637034,2637034,NaN,NaN
2637035,2637035,ümlauts,noun,en,"[{'links': [['ümlaut', 'ümlaut#English']], 'gl...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1644309,NaN,NaN,2637035,2637035,NaN,NaN
2637036,2637036,üntermensch,noun,en,"[{'categories': ['English countable nouns', 'E...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1644310,NaN,NaN,2637036,2637036,NaN,NaN


In [151]:
rough_df.drop(columns=['Unnamed: 0'], inplace=True)

In [152]:
rough_df.shape

(2637038, 51)

In [143]:
unique_vals = []
for c in rough_df.columns:
    vals = rough_df[c].unique()
    if len(vals) < 100:
        unique_vals.append((c, vals))


In [155]:
for v in unique_vals:
    print(v[0] + ', Length:' + str(len(v[1])))

pos, Length:30
lang_code, Length:2
etymology_number, Length:18
form_of, Length:4
hyphenation, Length:11
info_templates, Length:8
instances, Length:91
lang, Length:4
metonyms, Length:83
origin, Length:3
paronyms, Length:5
pos_title, Length:36
source, Length:2
tags, Length:26
troponyms, Length:15
wikidata, Length:2
alt_of, Length:2
redirects, Length:1
count, Length:17
ety_text, Length:1


In [ ]:
rough_df.drop(columns=['ety_text', 'redirects', 'alt_of', 'temp_entry_num'], inplace=True)

# Category Columns

### utils

In [108]:
def merge_values(col1, col2):
    if pd.isna(col1) and pd.isna(col2):
        return None
    elif pd.isna(col1):
        return col2
    elif pd.isna(col2):
            return col1
    elif col1 == col2:
         return col1
    else:
         return f"REVIEW: {col1} / {col2}"

- check for deletion
""" 

etymology_number, Length:18
form_of, Length:4 (del)
hyphenation, Length:11
info_templates, Length:8
instances, Length:91
metonyms, Length:83
paronyms, Length:5
pos_title, Length:36
tags, Length:26
troponyms, Length:15
wikidata, Length:2
alt_of, Length:2 (del)
redirects, Length:1 (del)
count, Length:17
ety_text, Length:1 (del)
"""

#### Form Of

In [158]:
form_of = cut_df_to_category(rough_df, 'form_of')

In [159]:
display(form_of)

,word_id,word,pos,form_of
378517,572304,Godself,pron,[{'word': 'God'}]
1190923,572304,Godself,pron,[{'word': 'God'}]
2127615,998267,oneself,pron,[{'word': 'the indefinite personal pronoun one'}]
2622461,1615919,xemself,pron,[{'word': 'xe'}]


In [170]:
rough_df.drop(columns=['form_of'], inplace=True)

In [162]:
forms_df = cut_df_to_category(rough_df, 'forms')

In [163]:
forms_df

,word_id,word,pos,forms
4,10,'alf,noun,"[{'form': ""'alves"", 'tags': ['plural']}]"
7,14,'ammer,noun,"[{'form': ""'ammers"", 'tags': ['plural']}]"
10,18,'aporth,noun,"[{'form': ""'aporths"", 'tags': ['plural']}]"
14,29,'arbour,noun,"[{'form': ""'arbours"", 'tags': ['plural']}]"
20,40,'Arry,noun,"[{'form': ""'Arrys"", 'tags': ['plural']}, {'for..."
...,...,...,...,...
2637030,1644300,überpopular,adj,"[{'form': 'more überpopular', 'tags': ['compar..."
2637031,1644306,übersexual,adj,"[{'form': 'more übersexual', 'tags': ['compara..."
2637032,1644306,übersexual,noun,"[{'form': 'übersexuals', 'tags': ['plural']}]"
2637034,1644308,ümlaut,noun,"[{'form': 'ümlauts', 'tags': ['plural']}]"


#### Alt Of

In [ ]:
alt_df = return_non_na(rough_df, 'alt_of')
alt_df

,word,pos,lang_code,senses,abbreviations,anagrams,antonyms,categories,coordinate_terms,derived,...,translations,troponyms,wikidata,wikipedia,lower,word_id,alt_of,temp_entry_num,tmpid,count
2462008,tag teamed,verb,en,"[{'links': [['tag team', 'tag team#English']],...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1405961,[{'word': 'tag-teamed'}],2462008,2462008,NaN


In [174]:
rough_df.drop(columns=['alt_of'], inplace=True)

### Wikdata

In [176]:
wikidata_df = return_non_na(rough_df, 'wikidata')
wikidata_df

,word,pos,lang_code,senses,abbreviations,anagrams,antonyms,categories,coordinate_terms,derived,...,tags,translations,troponyms,wikidata,wikipedia,lower,word_id,temp_entry_num,tmpid,count
232925,deconstructivist,noun,en,"[{'links': [['deconstructivism', 'deconstructi...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,['Q115206842'],NaN,deconstructivist,353009,356929,356929,1.0
1649685,deconstructivist,noun,en,"[{'links': [['deconstructivism', 'deconstructi...",NaN,NaN,NaN,"['English adjectives', 'English countable noun...",NaN,NaN,...,NaN,NaN,NaN,['Q115206842'],NaN,NaN,353009,1649685,1649685,NaN
1649686,deconstructivist,adj,en,[{'examples': [{'text': 'It’s a studied disrup...,NaN,NaN,NaN,"['English adjectives', 'English countable noun...",NaN,NaN,...,NaN,NaN,NaN,['Q115206842'],NaN,NaN,353009,1649686,1649686,NaN


In [177]:
rough_df.drop(columns=['wikidata'], inplace=True)

### Info Templates

In [ ]:
inf_df = cut_df_to_category(rough_df, 'info_templates')
inf_df

,word_id,word,pos,info_templates
408316,616901,have no business,verb,"[{'args': {'1': 'en', '2': '::-ing'}, 'name': ..."
798508,1215120,ride the coattails,verb,"[{'args': {'1': 'en', '2': ':of'}, 'name': '+o..."
925421,1407294,take kindly,verb,"[{'args': {'1': 'en', '2': ':to'}, 'name': '+o..."
1814053,563829,give weight,verb,"[{'args': {'1': 'en', '2': ':to'}, 'name': '+o..."
1852670,616874,have it bad,verb,"[{'args': {'1': 'en', '2': ':for'}, 'name': '+..."
1852696,616901,have no business,verb,"[{'args': {'1': 'en', '2': '::-ing'}, 'name': ..."
1872328,644127,hold a grudge,verb,"[{'args': {'1': 'en', '2': ':against'}, 'name'..."
2005515,838980,make a pass,verb,"[{'args': {'1': 'en', '2': ':at'}, 'name': '+o..."
2171064,1054666,pass through the hands,verb,"[{'args': {'1': 'en', '2': ':of'}, 'name': '+o..."
2311664,1215120,ride the coattails,verb,"[{'args': {'1': 'en', '2': ':of'}, 'name': '+o..."


In [188]:
def transform_df_to_col_table(origin_df, col_name, group_key='word_id'):
    dropped_df = origin_df[~origin_df[col_name].isna()]

    #words_df = dropped_df.loc[:, ['tmp_id', 'word']]
    words_df = dropped_df.loc[:, ['word_id', 'word']]
    df = dropped_df.loc[:, ['word_id', 'pos', col_name]]
    df[col_name] = df[col_name].apply(safe_eval)
    df = df.explode(col_name, ignore_index=True)
    
    df = pd.concat([df.drop(columns=col_name), pd.json_normalize(df[col_name])],axis=1)
    
    df = df.groupby(group_key, as_index=False).aggregate('first').reindex(columns=df.columns)
    if 'word' in df.columns:
        df = df.rename(columns={'word': col_name})
    
    df = df.merge(words_df, on='word_id')
    df_cols = df.columns.tolist()
    df_cols = df_cols[0:2] + ['word'] + df_cols[2:-1]
    df = df[df_cols]
    
    return df

In [189]:
col_name = 'info_templates'
inf_df = transform_df_to_col_table(rough_df,col_name)

In [191]:
inf_df.to_csv(Path(current_save_path, 'column_dfs', 'info_templates.csv'))

In [192]:
rough_df.drop(columns=['info_templates'], inplace=True)

### Compress Entries

In [207]:
grouped = rough_df.groupby(['word', 'lang']).agg(list).reset_index()

KeyboardInterrupt: 

In [ ]:
display(grouped)

In [209]:
rough_df.columns

Index(['word', 'pos', 'lang_code', 'senses', 'abbreviations', 'anagrams',
       'antonyms', 'categories', 'coordinate_terms', 'derived', 'descendants',
       'etymology_number', 'etymology_templates', 'etymology_texts', 'forms',
       'head_templates', 'holonyms', 'homophones', 'hypernyms', 'hyphenation',
       'hyphenations', 'hyponyms', 'inflection_templates', 'instances', 'lang',
       'meronyms', 'metonyms', 'notes', 'origin', 'paronyms', 'pos_title',
       'proverbs', 'related', 'sounds', 'source', 'synonyms', 'tags',
       'translations', 'troponyms', 'wikipedia', 'lower', 'word_id', 'tmpid',
       'count'],
      dtype='object')

In [208]:
rough_df['origin'].value_counts()

origin
raw_cb    1547243
pp_en      776472
pp_nl      313323
Name: count, dtype: int64

In [210]:
rough_df['source'].value_counts()

source
thesaurus    103
Name: count, dtype: int64

In [ ]:
new_cols = ['word_id', 'word', 'pos', 'l', 'dl','senses', 'abbreviations',
       'antonyms', 'categories', 'coordinate_terms', 'derived', 'descendants',
       'etymology_number', 'etymology_templates', 'etymology_texts', 'forms',
       'head_templates', 'holonyms', 'homophones', 'hypernyms', 'hyphenation',
       'hyphenations', 'hyponyms', 'inflection_templates', 'instances',
       'meronyms', 'metonyms', 'notes', 'origin', 'paronyms', 
       'proverbs', 'related', 'sounds', 'synonyms', 'tags',
       'translations', 'troponyms', 'srcf']

In [ ]:
dl_col =[] # dictionary language
s_col = [] # source col
for i, row in tqdm(rough_df.iterrows()):
    lang = row['lang']
    origin = row['origin']
    lang_code = row['lang_code']
    
    if lang in ['English', 'Dutch']:
        dl_col.append('wkt_en')
    elif lang in ['Engels', 'Nederlands']:
        dl_col.append('wkt_nl')
    if 'pp' in origin:
        s_col.append('pp')
    elif origin == 'raw_cb':
        s_col.append('r')
    if i%100000 == 0:
        print(row)
df = rough_df.loc[:, ['word_id', 'word', 'pos', 'lang_code','senses', 'abbreviations',
       'antonyms', 'categories', 'coordinate_terms', 'derived', 'descendants',
       'etymology_number', 'etymology_templates', 'etymology_texts', 'forms',
       'head_templates', 'holonyms', 'homophones', 'hypernyms', 'hyphenation',
       'hyphenations', 'hyponyms', 'inflection_templates', 'instances',
       'meronyms', 'metonyms', 'notes', 'paronyms',
       'proverbs', 'related', 'sounds', 'synonyms', 'tags',
       'translations', 'troponyms']]

df['wikl'] = dl_col
df['srcf'] = s_col

4806it [00:14, 580.05it/s]

word                                                                    '
pos                                                                symbol
lang_code                                                              en
senses                  [{'links': [['’', '’#English']], 'glosses': ['...
abbreviations                                                         NaN
anagrams                                                              NaN
antonyms                                                              NaN
categories                                                            NaN
coordinate_terms                                                      NaN
derived                                                               NaN
descendants                                                           NaN
etymology_number                                                      NaN
etymology_templates                                                   NaN
etymology_texts                       

104994it [00:16, 37159.88it/s]

word                                                         bespioneerde
pos                                                                  verb
lang_code                                                              nl
senses                  [{'glosses': ['enkelvoud verleden tijd van bes...
abbreviations                                                         NaN
anagrams                                                              NaN
antonyms                                                              NaN
categories              [{'name': 'Ontbrekend geluid', 'kind': 'other'...
coordinate_terms                                                      NaN
derived                                                               NaN
descendants                                                           NaN
etymology_number                                                      NaN
etymology_templates                                                   NaN
etymology_texts                       

207798it [00:19, 39097.49it/s]

...

2507317it [01:18, 40146.63it/s]

word                                                                 toof
pos                                                                  noun
lang_code                                                              en
senses                  [{'categories': ['English countable nouns', 'E...
abbreviations                                                         NaN
anagrams                                                              NaN
antonyms                                                              NaN
categories                                                            NaN
coordinate_terms                                                      NaN
derived                                                               NaN
descendants                                                           NaN
etymology_number                                                      NaN
etymology_templates                                                   NaN
etymology_texts                       

2609287it [01:21, 40179.84it/s]

word                                                         weatherdecks
pos                                                                  noun
lang_code                                                              en
senses                  [{'links': [['weatherdeck', 'weatherdeck#Engli...
abbreviations                                                         NaN
anagrams                                                              NaN
antonyms                                                              NaN
categories                                                            NaN
coordinate_terms                                                      NaN
derived                                                               NaN
descendants                                                           NaN
etymology_number                                                      NaN
etymology_templates                                                   NaN
etymology_texts                       

2637038it [01:21, 32194.18it/s]


In [220]:
df = df.loc[:, ['word_id', 'word', 'pos', 'lang_code', 'wikl', 'senses', 'abbreviations', 'antonyms',
       'categories', 'coordinate_terms', 'derived', 'descendants',
       'etymology_number', 'etymology_templates', 'etymology_texts', 'forms',
       'head_templates', 'holonyms', 'homophones', 'hypernyms', 'hyphenation',
       'hyphenations', 'hyponyms', 'inflection_templates', 'instances',
       'meronyms', 'metonyms', 'notes', 'paronyms', 'proverbs', 'related',
       'sounds', 'synonyms', 'tags', 'translations', 'troponyms', 
       'srcf']]

In [217]:
df = df.sort_values('word_id')

In [221]:
display(df.head())

,word_id,word,pos,lang_code,wikl,senses,abbreviations,antonyms,categories,coordinate_terms,...,notes,paronyms,proverbs,related,sounds,synonyms,tags,translations,troponyms,srcf
1089796,0,',symbol,en,wkt_en,"[{'links': [['’', '’#English']], 'glosses': ['...",NaN,NaN,['English entries with incorrect language head...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,r
1089795,0,',particle,en,wkt_en,"[{'links': [[""-'"", ""-'#English""]], 'glosses': ...",NaN,NaN,['English entries with incorrect language head...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,r
0,0,',symbol,en,wkt_en,"[{'links': [['’', '’#English']], 'glosses': ['...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pp
1089852,1,'a,verb,en,wkt_en,[{'categories': ['English terms with archaic s...,NaN,NaN,['English entries with incorrect language head...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,r
1089853,1,'a,pron,en,wkt_en,[{'categories': ['English terms with archaic s...,NaN,NaN,['English entries with incorrect language head...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,r


### Senses

In [206]:
sense_keys = []
test_senses = rough_df.iloc[0:10, :]

    

In [ ]:
rough_df.drop(columns=[], inplace=True)

In [197]:
senses_df = return_non_na(rough_df, 'senses')

In [198]:
senses_df.columns

Index(['word', 'pos', 'lang_code', 'senses', 'abbreviations', 'anagrams',
       'antonyms', 'categories', 'coordinate_terms', 'derived', 'descendants',
       'etymology_number', 'etymology_templates', 'etymology_texts', 'forms',
       'head_templates', 'holonyms', 'homophones', 'hypernyms', 'hyphenation',
       'hyphenations', 'hyponyms', 'inflection_templates', 'instances', 'lang',
       'meronyms', 'metonyms', 'notes', 'origin', 'paronyms', 'pos_title',
       'proverbs', 'related', 'sounds', 'source', 'synonyms', 'tags',
       'translations', 'troponyms', 'wikipedia', 'lower', 'word_id', 'tmpid',
       'count'],
      dtype='object')

In [200]:
cols = ['tmpid', 'word_id', 'word', 'pos', 'lang_code', 'lang', 'senses']

In [201]:
senses_df = senses_df.loc[:, cols]
senses_df

,tmpid,word_id,word,pos,lang_code,lang,senses
0,0,0,',symbol,en,English,"[{'links': [['’', '’#English']], 'glosses': ['..."
1,23,1,'a,verb,en,English,"[{'links': [['a', 'a#English']], 'raw_glosses'..."
2,24,3,'a'a,noun,en,English,"[{'links': [['aa', 'aa#English']], 'glosses': ..."
3,25,9,'akikikis,noun,en,English,"[{'links': [[""'akikiki"", ""'akikiki#English""]],..."
4,26,10,'alf,noun,en,English,"[{'links': [['Pronunciation spelling', 'pronun..."
...,...,...,...,...,...,...,...
2637033,2637033,1644307,übersexuals,noun,en,English,"[{'links': [['übersexual', 'übersexual#English..."
2637034,2637034,1644308,ümlaut,noun,en,English,[{'examples': [{'text': 'The Traveler said he ...
2637035,2637035,1644309,ümlauts,noun,en,English,"[{'links': [['ümlaut', 'ümlaut#English']], 'gl..."
2637036,2637036,1644310,üntermensch,noun,en,English,"[{'categories': ['English countable nouns', 'E..."


### Nyms

### etymology

In [124]:
etymology_df = rough_df.loc[:, ['tmpid', 'word', 'word_id', 'pos', 'lang_code', 'lang', 'senses', 'etymology_number', 'etymology_templates',
       'etymology_text', 'etymology_texts', 'count']]

In [125]:
et = etymology_df.loc[:, ['tmpid', 'word', 'etymology_text']]
et.rename(columns={'etymology_text': 'etymology_texts'}, inplace=True)

In [128]:
etymology_df = etymology_df.combine_first(et)

In [142]:
etymology_df = etymology_df.loc[:, ['tmpid', 'word', 'word_id', 'lang', 'etymology_number', 'etymology_templates', 'etymology_texts',  'count', 'lang_code', 'pos', 'senses']]

In [144]:
etymology_df.shape

(2637038, 11)

In [145]:
rough_df['etymology_texts'] = etymology_df['etymology_texts']
rough_df.drop(columns=['etymology_text'], inplace=True)

In [205]:
rough_df.to_csv(Path(current_save_path, 'rough_df_30-10-25.csv'))

In [ ]:
ety_df_list = []
etymology_df2 = etymology_df[~etymology_df['etymology_texts'].isna() | ~etymology_df['etymology_number'].isna() | ~etymology_df['etymology_templates'].isna()]